Image preprocessing steps I've used in this project:
1) Grayscaling
2) Binarization
3) Sharpening filter

In [1]:
# !pip install opencv-python-headless pytesseract Pillow

In [20]:
# Install the necessary libraries

import numpy as np
import cv2
from PIL import Image
import pytesseract
import os
import re

# For Windows: Set the path to the installed Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


# Function to preprocess the image by converting to grayscale and applying threshold
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply a binary threshold to make the text more clear
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    
    
    kernel_sharpen = np.array([[-1, -1, -1],
                           [-1,  9, -1],
                           [-1, -1, -1]])
    sharpened_img = cv2.filter2D(thresh, -1, kernel_sharpen)


    
    # Save and return the preprocessed image path
    processed_image_path = image_path.replace(".jpg", "_processed.jpg")
    cv2.imwrite(processed_image_path, sharpened_img)
    return processed_image_path

# Function to extract text from an image after preprocessing
def extract_text_from_image(image_path):
    # Preprocess the image (convert to grayscale and thresholding)
    processed_image_path = preprocess_image(image_path)
    
    # Open the processed image using PIL
    image = Image.open(processed_image_path)
    
    # Extract text using pytesseract
    text = pytesseract.image_to_string(image)
    return text

# Function to parse the price from the extracted text
def parse_price(text):
    price_pattern = r'Rs[\.\s]*[0-9]+[\./0-9]*'  # Refined price pattern
    match = re.search(price_pattern, text)
    return match.group(0) if match else "Price not found"

# Function to parse the quantity from the extracted text
def parse_quantity(text):
    quantity_pattern = r'[0-9]+\s?[kK]?[gG]' # Refined quantity pattern
    match = re.search(quantity_pattern, text)
    return match.group(0) if match else "Quantity not found"

# Main function to compare products based on their images
def compare_products(image1_path, image2_path):
    # Extract text from both images
    text1 = extract_text_from_image(image1_path)
    text2 = extract_text_from_image(image2_path)
    
    # Parse price and quantity for both products
    price1 = parse_price(text1)
    quantity1 = parse_quantity(text1)
    
    price2 = parse_price(text2)
    quantity2 = parse_quantity(text2)
    
    # Display the comparison results
    print("COMPARISON:")
    print()
    print("Product 1:")
    print(f"Price: {price1}")
    print(f"Quantity: {quantity1}\n")
    
    print("Product 2:")
    print(f"Price: {price2}")
    print(f"Quantity: {quantity2}\n")
    
    # Return a comparison dictionary (optional for further use)
    comparison_result = {
        'Product 1': {'Price': price1, 'Quantity': quantity1},
        'Product 2': {'Price': price2, 'Quantity': quantity2}
    }
    
    return comparison_result

# Paths to the uploaded images
image1_path = './assets/product1.jpg'
image2_path = './assets/product2.jpg'

# Ensure the images are available
print("Product 1 image exists:", os.path.exists(image1_path))
print("Product 2 image exists:", os.path.exists(image2_path))
print()

# Compare the products by extracting text and parsing price and quantity
comparison = compare_products(image1_path, image2_path)


Product 1 image exists: True
Product 2 image exists: True

COMPARISON:

Product 1:
Price: Rs 649.00
Quantity: 500g

Product 2:
Price: Rs 85
Quantity: 1kg



In [3]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
